<a href="https://colab.research.google.com/github/UriKialy/intro-to-deep-learning-pytorch-udacity/blob/main/Sentiment_RNN_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis.
>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words.

Here we'll use a dataset of movie reviews, accompanied by sentiment labels: positive or negative.

<img src="assets/reviews_ex.png" width=40%>

### Network Architecture

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the Word2Vec lesson. You can actually train an embedding with the Skip-gram Word2Vec model and use those embeddings as input, here. However, it's good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in the movie review data.

>**Finally, the LSTM outputs will go to a sigmoid output layer.** We're using a sigmoid function because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1.

We don't care about the sigmoid outputs except for the **very last one**; we can ignore the rest. We'll calculate the loss by comparing the output at the last time step and the training label (pos or neg).

---
### Load in and visualize the data

In [6]:
import os
import requests
import tarfile

# URLs for dataset download (hosted on Kaggle or public repositories)
dataset_url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# File paths
data_dir = "data"
dataset_path = os.path.join(data_dir, "aclImdb_v1.tar.gz")
extracted_dir = os.path.join(data_dir, "aclImdb")

# Ensure the data directory exists
os.makedirs(data_dir, exist_ok=True)

# Download the dataset if not already present
if not os.path.exists(dataset_path):
    print("Downloading IMDB dataset...")
    response = requests.get(dataset_url, stream=True)
    with open(dataset_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

# Extract the dataset if not already extracted
if not os.path.exists(extracted_dir):
    print("Extracting IMDB dataset...")
    with tarfile.open(dataset_path, "r:gz") as tar:
        tar.extractall(path=data_dir)

# Read reviews and labels
print("Loading reviews and labels...")

train_dir = os.path.join(extracted_dir, "train")

# Load positive and negative reviews
reviews = []
labels = []

for label in ["pos", "neg"]:  # Loop through positive and negative directories
    dir_path = os.path.join(train_dir, label)
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename), "r", encoding="utf-8") as file:
            reviews.append(file.read())
            labels.append("pos" if label == "pos" else "neg")  # Append "pos" or "neg"

# Debugging outputs
print(f"Total reviews loaded: {len(reviews)}")
print(f"Total labels loaded: {len(labels)}")
print(f"Unique labels: {set(labels)}")


# Convert to single strings
reviews = "\n".join(reviews)
labels = "\n".join(labels)


Loading reviews and labels...
Total reviews loaded: 25000
Total labels loaded: 25000
Unique labels: {'pos', 'neg'}


In [7]:
print(len(reviews),len(labels))

33151740 99999


In [9]:
# Ensure reviews and labels have the same length
min_length = min(len(reviews), len(labels))
print(min_length)
# Truncate both to the same length
reviews = reviews[:min_length]
labels = labels[:min_length]

# Print the preprocessed samples
#print(reviews[:min_length])  # Print reviews up to the truncated length
#print()
#print(labels[:min_length])  # Print labels up to the truncated length


99999


## Data pre-processing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. Here are the processing steps, we'll want to take:
>* We'll want to get rid of periods and extraneous punctuation.
* Also, you might notice that the reviews are delimited with newline characters `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter.
* Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [10]:
from string import punctuation

print(punctuation)

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [11]:
# Split the text into words based on whitespace (spaces, newlines, etc.)
words = all_text.split()  # Splits on any whitespace character (including newline, space, etc.)

# Print the first 20 words to check
print(words[:20])  # Print the first 20 words


['i', 'saw', 'it', 'at', 'the', 'legacy', 'theater', 'in', 'the', 'joseph', 'smith', 'memorial', 'building', 'in', 'salt', 'lake', 'city', 'this', 'morning', 'im']


In [12]:
words[:30]

['i',
 'saw',
 'it',
 'at',
 'the',
 'legacy',
 'theater',
 'in',
 'the',
 'joseph',
 'smith',
 'memorial',
 'building',
 'in',
 'salt',
 'lake',
 'city',
 'this',
 'morning',
 'im',
 'going',
 'to',
 'assume',
 'that',
 'ones',
 'level',
 'of',
 'enjoyment',
 'during',
 'this']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`.

In [14]:
from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [15]:
from collections import Counter

# Count how often each word appears
counts = Counter(words)

# Create a sorted list of words by appearance frequency
vocab = sorted(counts, key=counts.get, reverse=True)

# Create a mapping from word to integer
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# Tokenize the reviews
reviews_ints = []
for review in reviews_split:
    # Tokenize and handle words that are in vocab_to_int
    tokenized_review = [vocab_to_int[word] for word in review.split() if word in vocab_to_int]
    if tokenized_review:  # Only append non-empty tokenized reviews
        reviews_ints.append(tokenized_review)

# Check if reviews_ints aligns with labels
print(f"Number of tokenized reviews: {len(reviews_ints)}")
print(f"Number of original reviews: {len(reviews_split)}")
assert len(reviews_ints) == len(reviews_split), "Mismatch between tokenized reviews and original reviews"

# Check if vocab_to_int is working fine
print("Sample tokenized review (first review):", reviews_ints[0] if reviews_ints else "No reviews available")


Number of tokenized reviews: 73
Number of original reviews: 73
Sample tokenized review (first review): [9, 182, 8, 30, 1, 893, 470, 6, 1, 627, 628, 1492, 894, 6, 895, 629, 379, 11, 896, 135, 158, 5, 1493, 10, 380, 471, 4, 897, 141, 11, 20, 77, 1494, 27, 313, 22, 380, 471, 4, 898, 4, 1495, 1496, 13, 142, 10, 1497, 8, 12, 44, 56, 95, 56, 630, 2, 59, 3, 381, 631, 36, 23, 110, 5, 895, 629, 379, 8, 7, 3, 224, 5, 54, 8, 6, 1, 893, 470, 9, 26, 172, 69, 6, 3, 1498, 470, 14, 314, 14, 251, 472, 473, 472, 2, 1499, 6, 61, 474, 1500, 899, 36, 1, 632, 41, 1501, 73, 86, 475, 633, 46, 70, 10, 135, 900, 2, 1502, 627, 14, 3, 1503, 9, 12, 476, 6, 477, 1504, 2, 12, 901, 5, 902, 96, 3, 382, 1505, 9, 1506, 7, 903, 201, 58, 1507, 37, 1508, 120, 1, 20, 904, 315, 135, 383, 10, 11, 15, 7, 905, 6, 477, 900, 1509, 1510, 202, 1, 1511, 36, 23, 26, 1512, 478, 906, 8, 37, 89, 479, 23, 384, 627, 628, 47, 21, 28, 634, 3, 635, 57]


**Test your code**

As a text that you've implemented the dictionary correctly, print out the number of unique words in your vocabulary and the contents of the first, tokenized review.

In [16]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  4142

Tokenized review: 
 [[9, 182, 8, 30, 1, 893, 470, 6, 1, 627, 628, 1492, 894, 6, 895, 629, 379, 11, 896, 135, 158, 5, 1493, 10, 380, 471, 4, 897, 141, 11, 20, 77, 1494, 27, 313, 22, 380, 471, 4, 898, 4, 1495, 1496, 13, 142, 10, 1497, 8, 12, 44, 56, 95, 56, 630, 2, 59, 3, 381, 631, 36, 23, 110, 5, 895, 629, 379, 8, 7, 3, 224, 5, 54, 8, 6, 1, 893, 470, 9, 26, 172, 69, 6, 3, 1498, 470, 14, 314, 14, 251, 472, 473, 472, 2, 1499, 6, 61, 474, 1500, 899, 36, 1, 632, 41, 1501, 73, 86, 475, 633, 46, 70, 10, 135, 900, 2, 1502, 627, 14, 3, 1503, 9, 12, 476, 6, 477, 1504, 2, 12, 901, 5, 902, 96, 3, 382, 1505, 9, 1506, 7, 903, 201, 58, 1507, 37, 1508, 120, 1, 20, 904, 315, 135, 383, 10, 11, 15, 7, 905, 6, 477, 900, 1509, 1510, 202, 1, 1511, 36, 23, 26, 1512, 478, 906, 8, 37, 89, 479, 23, 384, 627, 628, 47, 21, 28, 634, 3, 635, 57]]


### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively, and place those in a new list, `encoded_labels`.

In [17]:
import numpy as np
# 1=positive, 0=negative label conversion
lables_split=labels.split('\n')
encoded_labels = np.array([1 if label=='positive' else  0 for label in labels.split()])

### Removing Outliers

As an additional pre-processing step, we want to make sure that our reviews are in good shape for standard processing. That is, our network will expect a standard input text size, and so, we'll want to shape our reviews into a specific length. We'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.

<img src="assets/outliers_padding_ex.png" width=40%>

Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.

In [18]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 986


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. We'll have to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

> **Exercise:** First, remove *any* reviews with zero length from the `reviews_ints` list and their corresponding label in `encoded_labels`.

In [19]:
print(len(review_lens),len(encoded_labels))

65 25000


In [20]:
# Find indices of non-zero length reviews
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# Remove zero-length reviews and their corresponding labels using the same indices
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = [encoded_labels[ii] for ii in non_zero_idx]  # This must be a list, not a NumPy array

# Check lengths after filtering
print('Number of reviews after removing outliers: ', len(reviews_ints))
print('Number of labels after removing outliers: ', len(encoded_labels))


Number of reviews after removing outliers:  73
Number of labels after removing outliers:  73


---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network.
* The data should come from `review_ints`, since we want to feed integers to the network.
* Each row should be `seq_length` elements long.
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`.
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is:
```
[117, 18, 128]
```
The resultant, padded sequence should be:

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [21]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's
        or truncated to the input seq_length.
    '''
    features = []
    # Iterate over each review
    for review in reviews_ints:
        # If the review is already the desired length
        if len(review) == seq_length:
            features.append(review)
        # If the review is longer than the desired length, truncate it
        elif len(review) > seq_length:
            features.append(review[:seq_length])
        # If the review is shorter than the desired length, pad it with 0's
        else:
            # Calculate how many zeros to add
            padding = [0] * (seq_length - len(review))
            features.append(padding + review)

    features = np.array(features)
    return features


In [22]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches
print(features[:30,:10])

[[   0    0    0    0    0    0    0    0    0    0]
 [  11   12    3  636  907  143   40    1  908  637]
 [   8   12  924    5   27    1  650  651    5    1]
 [1600   23  492   10  187  493  260  945 1601  946]
 [  15   91   27    3  163 1721    5   54    1  100]
 [   0    0    0    0    0    0    0    0    0    0]
 [1969    6  138 1970 1971    2 1972 1973   63    4]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [ 245    1  174    4    3  119    7    3 2041  742]
 [   0    0    0    0    0    0    0    0    0    0]
 [  46   25   51  125  743   14    5  179    9 2097]
 [   9   68   81   11  111   24   45 1108    3  755]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0

## Training, Validation, Test

With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets.
* You'll need to create sets for the features and the labels, `train_x` and `train_y`, for example.
* Define a split fraction, `split_frac` as the fraction of data to **keep** in the training set. Usually this is set to 0.8 or 0.9.
* Whatever data is left will be split in half to create the validation and *testing* data.

In [23]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx=int(len(features)*split_frac)
train_x,remianing_x=features[:split_idx],features[split_idx:]
train_y,remaining_y=encoded_labels[:split_idx],encoded_labels[split_idx:]

test_idx=int(len(remianing_x)*0.5)
val_x,test_x=remianing_x[:test_idx],remianing_x[test_idx:]
val_y,test_y=remaining_y[:test_idx],remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(58, 200) 
Validation set: 	(7, 200) 
Test set: 		(8, 200)


**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200)
Validation set: 	(2500, 200)
Test set: 		  (2500, 200)
```

---
## DataLoaders and Batching

After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)
```

This is an alternative to creating a generator function for batching our data into full batches.

In [24]:
reviews = []  # Ensure reviews is a list
labels = []   # Ensure labels is a list

for label in ["pos", "neg"]:
    dir_path = os.path.join(train_dir, label)
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename), "r", encoding="utf-8") as file:
            reviews.append(file.read())  # Append review content to the list
            labels.append(label)         # Append the label ('pos' or 'neg') to the list


In [25]:
import os
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

# Example: Assuming you already have the raw text data
# Let's assume `reviews` is the list of reviews and `labels` is the list of corresponding labels

# Step 1: Tokenize the reviews (if not already done)
# Create a vocabulary and map words to integers
all_text = ' '.join(reviews)  # Join all reviews into one big string
words = all_text.split()  # Split into individual words
counts = Counter(words)  # Count the frequency of each word
vocab = sorted(counts, key=counts.get, reverse=True)  # Sort by frequency
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}  # Mapping from word to integer

# Tokenize the reviews using the vocab_to_int dictionary
train_x_tokenized = [[vocab_to_int[word] for word in review.split() if word in vocab_to_int] for review in reviews]

# Step 2: Convert labels to integers (0 for negative, 1 for positive)
encoded_labels = np.array([1 if label == 'pos' else 0 for label in labels])

# Step 3: Split data into training, validation, and test sets
train_x, val_x, test_x = train_x_tokenized[:20000], train_x_tokenized[20000:22500], train_x_tokenized[22500:]
train_y, val_y, test_y = encoded_labels[:20000], encoded_labels[20000:22500], encoded_labels[22500:]

# Step 4: Padding sequences to the same length
def pad_sequences(sequences, max_len):
    return [seq + [0] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len] for seq in sequences]

# Find the maximum length of a review to use for padding
max_len = max([len(review) for review in train_x])

# Pad all sequences to have the same length
train_x = pad_sequences(train_x, max_len)
val_x = pad_sequences(val_x, max_len)
test_x = pad_sequences(test_x, max_len)

# Step 5: Create Tensor datasets
train_data = TensorDataset(torch.tensor(train_x, dtype=torch.long), torch.tensor(train_y, dtype=torch.long))
valid_data = TensorDataset(torch.tensor(val_x, dtype=torch.long), torch.tensor(val_y, dtype=torch.long))
test_data = TensorDataset(torch.tensor(test_x, dtype=torch.long), torch.tensor(test_y, dtype=torch.long))

# Step 6: Create DataLoaders
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

# Print the number of batches in each DataLoader
print(f"Train loader batches: {len(train_loader)}")
print(f"Validation loader batches: {len(valid_loader)}")
print(f"Test loader batches: {len(test_loader)}")


Train loader batches: 400
Validation loader batches: 50
Test loader batches: 50


In [26]:
# obtain one batch of training data
sample_x, sample_y = next(iter(train_loader))

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 2470])
Sample input: 
 tensor([[     8,    128,     10,  ...,      0,      0,      0],
        [   322, 133976,     16,  ...,      0,      0,      0],
        [   324,    290,     15,  ...,      0,      0,      0],
        ...,
        [   550,     58,     67,  ...,      0,      0,      0],
        [   322,      4,      1,  ...,      0,      0,      0],
        [    55,      6,     25,  ...,      0,      0,      0]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
        1, 1])


---
# Sentiment Network with PyTorch

Below is where you'll define the network.

<img src="assets/network_diagram.png" width=40%>

The layers are as follows:
1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) that converts our word tokens (integers) into embeddings of a specific size.
2. An [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

We need to add an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

We'll create an [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, your network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships.

> **Exercise:** Complete the `__init__`, `forward`, and `init_hidden` functions for the SentimentRNN model class.

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [27]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [28]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        # define all layers


    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden


## Instantiate the network

Here, we'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `hidden_dim`: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM layers in the network. Typically between 1-3

> **Exercise:** Define the model  hyperparameters.


In [29]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 3

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(280618, 400)
  (lstm): LSTM(400, 256, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


---
## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. You can also add code to save a model by name.

>We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [30]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [31]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.684088... Val Loss: 0.912759
Epoch: 1/4... Step: 200... Loss: 0.662952... Val Loss: 0.989236
Epoch: 1/4... Step: 300... Loss: 0.656945... Val Loss: 0.938348
Epoch: 1/4... Step: 400... Loss: 0.688335... Val Loss: 0.971448
Epoch: 2/4... Step: 500... Loss: 0.625289... Val Loss: 0.933536
Epoch: 2/4... Step: 600... Loss: 0.697756... Val Loss: 0.940732
Epoch: 2/4... Step: 700... Loss: 0.704375... Val Loss: 0.986674
Epoch: 2/4... Step: 800... Loss: 0.657931... Val Loss: 0.941590
Epoch: 3/4... Step: 900... Loss: 0.669762... Val Loss: 1.004059
Epoch: 3/4... Step: 1000... Loss: 0.650716... Val Loss: 0.986236
Epoch: 3/4... Step: 1100... Loss: 0.628599... Val Loss: 0.944866
Epoch: 3/4... Step: 1200... Loss: 0.662891... Val Loss: 0.997624
Epoch: 4/4... Step: 1300... Loss: 0.693860... Val Loss: 0.960336
Epoch: 4/4... Step: 1400... Loss: 0.733690... Val Loss: 1.014971
Epoch: 4/4... Step: 1500... Loss: 0.686235... Val Loss: 1.013131
Epoch: 4/4... Step: 1600... Loss: 

---
## Testing

There are a few ways to test your network.

* **Test data performance:** First, we'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, we'll see if we can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [32]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()

    # get predicted outputs
    output, h = net(inputs, h)

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.928
Test accuracy: 0.000


### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!
    
> **Exercise:** Write a `predict` function that takes in a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!
* You can use any functions that you've already defined or define any helper functions you want to complete `predict`, but it should just take in a trained net, a text review, and a sequence length.


In [33]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [34]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

In [36]:
def predict(net, test_review, sequence_length=200):

    net.eval()

    # tokenize review
    test_ints = tokenize_review(test_review)

    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)

    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)

    batch_size = feature_tensor.size(0)

    # initialize hidden state
    h = net.init_hidden(batch_size)

    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()

    # get the output from the model
    output, h = net(feature_tensor, h)

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))

    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [37]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [38]:
# call function
# try negative and positive reviews!
seq_length=200
predict(net, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.592256
Positive review detected!


### Try out test_reviews of your own!

Now that you have a trained model and a predict function, you can pass in _any_ kind of text and this model will predict whether the text has a positive or negative sentiment. Push this model to its limits and try to find what words it associates with positive or negative.

Later, you'll learn how to deploy a model like this to a production environment so that it can respond to any kind of user data put into a web app!